In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from datetime import datetime
from datetime import date
from datetime import time

In [ ]:
df = pd.read_csv('../input/ecommerce-data/data.csv',encoding= 'cp874')
df

In [ ]:
df_time = df.dropna()
df_time = df_time[df_time['Quantity']>0]
df_time = df_time[df_time['UnitPrice']>0]
df_1 = df_time[~df_time['StockCode'].str.contains('POST')]
df_1

In [ ]:
df_t = df_1.loc[:,['CustomerID','InvoiceNo','InvoiceDate']]
df_t['InvoiceDate'] = pd.to_datetime(df_t.InvoiceDate)
df_t.info()

In [ ]:
df_t = df_t.drop_duplicates()

In [ ]:
wd = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
t = []
s = []
for i in df_t.InvoiceDate :
    w = i.weekday()
    s.append(wd[w])
    if w in [0,1,2,3,4] :
        t.append('Weekday')
    else :
        t.append('Weekend')


In [ ]:
df_t['DoW'] = s
df_t['NofWeek'] = t
df_t

In [ ]:
df_t = df_t.reset_index()
df_t

In [ ]:
df_t = df_t.drop(['index'],axis=1)
df_t

In [ ]:
from sklearn.preprocessing import OneHotEncoder
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_col = pd.DataFrame(OH_encoder.fit_transform(df_t[['NofWeek']]),columns=['Per_weekday','Per_weekend'])
OH_col

In [ ]:
df_t1 = pd.concat([df_t,OH_col],axis=1)
df_t1

In [ ]:
g = df_t1.groupby('CustomerID').sum()
g

In [ ]:
g['Count'] = g.sum(axis=1)
g['Percent_Weekday'] = ((g.Per_weekday / g.Count)*100).round(2)
g['Percent_Weekend'] = ((g.Per_weekend / g.Count)*100).round(2)
g

In [ ]:
g = g.reset_index()
g = g.loc[:,['CustomerID','Percent_Weekday','Percent_Weekend']]
g

In [ ]:
df_timing = df_t.loc[:,['CustomerID','InvoiceNo','InvoiceDate']]
time = []
for i in df_timing.InvoiceDate :
    time.append(i.hour)
df_timing['Time'] = time
df_timing

In [ ]:
t = []
for i in df_timing.Time :
    if i < 11 :
        t.append('Morning')
    elif i < 14 :
        t.append('Noon')
    elif i <17 :
        t.append('Afternoon')
    else:
        t.append('Evening')
df_timing['Timing'] = t
df_timing

In [ ]:
OH_timing = pd.DataFrame(OH_encoder.fit_transform(df_timing[['Timing']]),columns=['per_morning','per_noon','per_afternoon','per_evening'])
OH_timing

In [ ]:
OH_timing = OH_timing.rename(columns = {'per_morning':'Afternoon','per_noon':'Evening','per_afternoon':'Morning','per_evening':'Noon'})
OH_timing

In [ ]:
class_names = list(OH_encoder.categories_)

In [ ]:
class_names

In [ ]:
df_timing['Time'] = df_timing['Time'].astype('category')
df_timing.info()

In [ ]:
df_t2 = pd.concat([df_timing,OH_timing],axis=1)
df_t2

In [ ]:
c = df_t2.groupby('CustomerID').sum()
c

In [ ]:
c['Count'] = c.sum(axis=1)
c['%_morning'] = ((c['Morning'] / c['Count'])*100).round(2)
c['%_noon'] = ((c['Noon'] / c['Count'])*100).round(2)
c['%_afternoon'] = ((c['Afternoon'] / c['Count'])*100).round(2)
c['%_evening'] = ((c['Evening'] / c['Count'])*100).round(2)
c

In [ ]:
c = c.reset_index()
c = c.loc[:,['CustomerID','%_morning','%_noon','%_afternoon','%_evening']]
c

In [ ]:
df_finish = g.merge(c)
df_finish

In [ ]:
df_finish['CustomerID'] = df_finish['CustomerID'].astype('category')
df_finish.describe()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,silhouette_samples

In [ ]:
X = ['Percent_Weekday', 'Percent_Weekend', '%_morning','%_noon', '%_afternoon', '%_evening']

In [ ]:
df_input = df_finish[X]
df_input

In [ ]:
model = KMeans(n_clusters = 5)
y_kmeans = model.fit_predict(df_input)

In [ ]:
df_finish['Cluster'] = y_kmeans
df_finish

In [ ]:
df_finish.Cluster.value_counts()

In [ ]:
T = df_finish.groupby('Cluster').mean()
T['Count'] = df_finish.Cluster.value_counts()
T

In [ ]:
T['Group Name'] = ['Weekday, Morning','Weekday, Morning-Noon-Afternoon','Weekday, Noon','Weekend, Noon-Afternoon','Weekday,Afternoon']
T

Look at the customer cluster 0 . They usually buy the product on Weekday and in the morning.

In [ ]:
df_finish[df_finish['Cluster'] == 0 ]